<h1>Feed Forward Neural Network</h1>

<p>This is a very simple feed forward neural network that uses the relu activation function and gradient descent. The purpose of this project is to demonstrate an understanding of how neural networks work, as such the program is implemented from scratch in Lua. Lua was chosen specifically because it is minimalistic and so things as simple as reading a csv file had to be made almost from scratch. Although many better frameworks for building models exist this one has the advantage of running very quickly and being easy to use.</p>
<p>Two external libraries included in the repo are required to run the project. The xeus-lua kernel was used to run this jupyter notebook.</p>

<h2>Basic Functions</h2>

In [1]:
Object = require('classic')
require('DataFrame')

In [2]:

function dot(x,y)
    sum = 0
    for i = 1, #x do sum = sum + (x[i] * y[i]) end

    return sum
end

function relu(x)
    local out = {}
    for i = 1, #x do
        table.insert(math.max(x[i],0))
    end
    return out
end
function absoluteError(x,y)

    local sum = 0
    for i = 1, #x do
        sum = sum + math.abs(x[i] - y[i])
    end
    return sum
end

function meanAbsoluteError(x,y)
    local out = {}
    for i= 1, #x do
        table.insert(out, absoluteError(x[i], y[i]))
    end
    local sum = 0
    for i= 1, #out do
        sum = sum + out[i]
    end
    return sum / #out
end
    

In [3]:
x = DataFrame('input.csv',','):rows()
y = DataFrame('target.csv',','):rows()
table.remove(y,1)

<h2>Helper Classes</h2>

In [4]:
Neuron = Object:extend()
function Neuron:new(size)
    self.weights = {}
    for i = 1, size do
        table.insert(self.weights, math.random(0, 200) / 100) 
    end
    self.bias =  math.random(0, 200) / 100
end

function Neuron:copy()
    local other = Neuron(#self.weights)
    for i = 1, #self.weights do
        other.weights[i] = self.weights[i]
    end
    other.bias = self.bias
    return other
end

function Neuron:__tostring()
    local w = "Weights: ["
    for i=1, #self.weights do
        w = w ..  self.weights[i] .. ' '
    end
    w = w ..  '] '
  return  w .. "Bias: " .. self.bias
end

Layer = Object:extend()

function Layer:new(inputs, neurons)
    self.n = {}
    for i =1, neurons do
        table.insert(self.n,Neuron(inputs))
    end
end

function Layer:copy()
    local other = Layer(#self.n[1].weights, #self.n)
    for i =1, #self.n do
         other.n[i] = self.n[i]:copy()
    end
    return other
end

function Layer:__tostring()
    local w = "Layer: "
    for i=1, #self.n do
        w = w .. self.n[i]
    end
  return  w
end

function Layer:print()
    local w = "Layer: "
    for i=1, #self.n do
        print(self.n[i])
    end
end


layer = Layer(4,2)



<h2>Feed Forward Neural Net Class</h2>

In [5]:
FeedForward = Object:extend()
function FeedForward:new(inputs, layers )
    self.act = relu
    self.l = {}
    table.insert(self.l, Layer(inputs, layers[1]))
    for i = 2, #layers do
        table.insert(self.l, Layer(#self.l[i-1].n, layers[i]))
    end
    self.history = {self.l}
    
end

function FeedForward:copy_net(l)
    local other = {}
    for i = 1, #l do
        table.insert(other, l[i]:copy()) 
    end
    return other
end  

function FeedForward:print()
    for i=1, #self.l do
        print("Layer: " .. i)
        self.l[i]:print()
        print('\n')
    end
end

function FeedForward:runLayer(layer, inputs)
    local out = {}
    for i = 1, #layer.n do
        table.insert(out, dot(layer.n[1].weights, inputs))
    end
    return out
end

function FeedForward:forward(inputs)
    local out = {}
    for j=1, #inputs do
        local feed = inputs[j]
        for i =1, #self.l do

            feed = self:runLayer(self.l[i], feed)

        end
        table.insert(out, feed)
    end
    return out
end



function FeedForward:update(x,y)
    local mae = meanAbsoluteError(x,y)
    for i = 1, #self.l do
        for j = 1, #self.l[i].n do
            for k = 1, #self.l[i].n[j].weights do
                self.l[i].n[j].weights[k] = self.l[i].n[j].weights[k] + (math.random(-100,100) / 100) * mae
            end
        end
    end
end 

function FeedForward:train(x,y, iters)
    local mae = meanAbsoluteError(self:forward(x),y)
    print(mae)
    
    local best = self:copy_net(self.l)
    for i = 1, iters do
        
        self:update(self:forward(x),y)
        if meanAbsoluteError(self:forward(x),y) < mae then
            mae = meanAbsoluteError(self:forward(x),y)
            print('Improved iter = ' .. i.. ' MAE = ' .. mae)
            best = self:copy_net(self.l)
            table.insert(self.history, self.l)
        else
            
            self.l = self:copy_net(best)
        end
        
        
    end
    print(meanAbsoluteError(self:forward(x),y))
end
    



<h2>Running the Model</h2>

In [6]:
nn = FeedForward(2, {2,1})

In [7]:
x = DataFrame('input.csv',','):rows()
y = DataFrame('target.csv',','):rows()
table.remove(y,1)
print(nn:forward({x[2]}))
nn:train(x,y,1000)


{ { -7.0818530684175 } } 
49.194858103192 
Improved iter = 18 MAE = 37.393284869514 
Improved iter = 68 MAE = 7.1839953106567 
Improved iter = 95 MAE = 3.3572719062279 
Improved iter = 155 MAE = 1.9920638012829 
Improved iter = 196 MAE = 1.5205563361569 
Improved iter = 241 MAE = 0.16051216921696 
Improved iter = 299 MAE = 0.13997372278404 
Improved iter = 401 MAE = 0.11626820905704 
Improved iter = 467 MAE = 0.049763201782986 
Improved iter = 612 MAE = 0.030419360935125 
Improved iter = 763 MAE = 0.025560509425603 
Improved iter = 826 MAE = 0.013276465189396 
Improved iter = 947 MAE = 0.0029374848796395 
0.0029374848796395 


<p>Above the model is demonstrated to run properly for a toy data set and <i>mean absolute error</i> decreases over time. 